In [1]:
import pandas as pd

from surprise import SVD
from surprise import NormalPredictor

from surprise import Dataset
from surprise import Reader

from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

from helpers import *

In [2]:
data = load_data("data/data_train.csv")
data.head()

,user,movie,rating
0,r44,c1,4
1,r61,c1,3
2,r67,c1,4
3,r72,c1,3
4,r86,c1,5


In [3]:
dataset = Dataset.load_from_df(data[['user', 'movie', 'rating']], Reader(rating_scale=(1, 5)))

In [13]:
param_grid = {'n_epochs': [45, 60], 'lr_all': [0.007, 0.010], 'reg_all': [0.1, 0.4]}
# param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(dataset)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.99425282924499
{'n_epochs': 60, 'lr_all': 0.007, 'reg_all': 0.1}


In [17]:
algo = SVD(n_factors=8, n_epochs=25, lr_all=0.081, reg_all=0.081)
cross_validate(algo, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0491  1.0481  1.0539  1.0492  1.0476  1.0496  0.0022  
MAE (testset)     0.8554  0.8532  0.8586  0.8552  0.8528  0.8550  0.0020  
Fit time          25.91   25.90   25.65   25.25   25.95   25.73   0.26    
Test time         1.85    3.81    3.11    3.21    3.28    3.05    0.65    


{'test_rmse': array([1.04913607, 1.04811165, 1.0538631 , 1.04923372, 1.04764784]),
 'test_mae': array([0.8553582 , 0.85321343, 0.85855183, 0.85524081, 0.8527816 ]),
 'fit_time': (25.90652084350586,
  25.902015209197998,
  25.6458261013031,
  25.25154709815979,
  25.953684329986572),
 'test_time': (1.8486571311950684,
  3.8141250610351562,
  3.1091911792755127,
  3.213458776473999,
  3.277639150619507)}

In [15]:
gen_submission(r'data/submission_svd_imp_3.csv', algo)